<a href="https://colab.research.google.com/github/IoannisDem/ViT-explainability/blob/main/ViT_DINO_video.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install -q transformers
!pip install datasets
!pip install torchvision

## Import library

In [2]:
from transformers import ViTFeatureExtractor, ViTModel, ViTImageProcessor
from datasets import load_dataset
import torch
from torch import nn
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
import numpy as np
import os
import torchvision
import skimage.io
from skimage.measure import find_contours
import random
import colorsys
import cv2
import sys
from PIL import Image

## Load model

In [3]:
model = ViTModel.from_pretrained('facebook/dino-vits8', add_pooling_layer=False) # load DINO model
dataset = load_dataset("huggingface/cats-image") # load cat image
processor = ViTImageProcessor.from_pretrained('facebook/dino-vits8', size=480) # use image processor for DINO model and resize to 480x480

config.json:   0%|          | 0.00/452 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/86.8M [00:00<?, ?B/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating test split: 0 examples [00:00, ? examples/s]

preprocessor_config.json:   0%|          | 0.00/244 [00:00<?, ?B/s]

## Load dataset/video

In [4]:
path = '/content/final.mp4'

In [5]:
vidcap = cv2.VideoCapture(path)
success, image = vidcap.read()
count = 0
while success:
  cv2.imwrite("/content/raw/frame%d.jpg" % count, image)     # save frame as JPEG file
  success, image = vidcap.read()
  print('Read a new frame: ', success)
  count += 1

Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame

In [6]:
z = 'frame123.jpg'
z.split('frame')[1].split('.')[0]

'123'

## Create attention maps

In [7]:
import glob
import os

In [8]:
imgs = glob.glob('/content/raw/*.jpg')
len(imgs)

648

In [10]:
patch_size = 8
output_dir = '/content/processed'

In [11]:
for image_path in imgs:
  print(image_path)
  count = int(image_path.split('frame')[1].split('.')[0])
  print(count)
  image = Image.open(image_path)
  processed_image = processor(images=image, return_tensors="pt").pixel_values
  outputs = model(processed_image, output_attentions=True, interpolate_pos_encoding=True)

  thres = 0.6
  w_featmap = processed_image.shape[-2]//patch_size
  h_featmap = processed_image.shape[-1]//patch_size

  last_layer = outputs.attentions[-1]
  nh = last_layer.shape[1]

  keep_attention  = last_layer[0, :, 0, 1:].reshape(nh, -1)

  if thres is not None:
    val, idx = torch.sort(keep_attention)
    val /= torch.sum(val, dim=1, keepdim=True)

    cumval = torch.cumsum(val, dim=1)
    th_attn = cumval > (1 - thres)
    idx2 = torch.argsort(idx)
    for head in range(nh):
        th_attn[head] = th_attn[head][idx2[head]]
    th_attn = th_attn.reshape(nh, w_featmap, h_featmap).float()
    th_attn = nn.functional.interpolate(th_attn.unsqueeze(0), scale_factor=patch_size, mode="nearest")[0].cpu().numpy()

    attentions = keep_attention.reshape(nh, w_featmap, h_featmap)
    attentions = nn.functional.interpolate(attentions.unsqueeze(0),
                                       scale_factor=patch_size,
                                       mode="nearest")[0].detach().cpu().numpy()

    # save attention heatmaps
    for j in range(nh):
      fname = os.path.join(output_dir, str(count) + "attn-headd" + str(j) + ".png")
      plt.imsave(fname=fname, arr=attentions[j], format='png')
      print(f"{fname} saved.")

Streaming output truncated to the last 5000 lines.
/content/raw/frame495.jpg
495
/content/processed/495attn-headd0.png saved.
/content/processed/495attn-headd1.png saved.
/content/processed/495attn-headd2.png saved.
/content/processed/495attn-headd3.png saved.
/content/processed/495attn-headd4.png saved.
/content/processed/495attn-headd5.png saved.
/content/raw/frame532.jpg
532
/content/processed/532attn-headd0.png saved.
/content/processed/532attn-headd1.png saved.
/content/processed/532attn-headd2.png saved.
/content/processed/532attn-headd3.png saved.
/content/processed/532attn-headd4.png saved.
/content/processed/532attn-headd5.png saved.
/content/raw/frame410.jpg
410
/content/processed/410attn-headd0.png saved.
/content/processed/410attn-headd1.png saved.
/content/processed/410attn-headd2.png saved.
/content/processed/410attn-headd3.png saved.
/content/processed/410attn-headd4.png saved.
/content/processed/410attn-headd5.png saved.
/content/raw/frame330.jpg
330
/content/processed/

## Zip folder to download

In [16]:
import tarfile
def make_tar_gz(source_folder, output_filename):
    with tarfile.open(output_filename, "w:gz") as tar:
        tar.add(source_folder, arcname=os.path.basename(source_folder))

In [18]:
# Example usage
source_folder = "/content/processed"
output_filename = "/content/processed_small.tar.gz"

make_tar_gz(source_folder, output_filename)

## Create video

In [2]:
import tarfile

def unzip_tar(tar_filename, destination_folder):
    with tarfile.open(tar_filename, 'r') as tar:
        tar.extractall(destination_folder)

In [3]:
# Example usage
tar_filename = "/content/processed_small.tar.gz"
destination_folder = "/content/processed.tar.gz"

unzip_tar(tar_filename, destination_folder)

In [30]:
# select_video = sorted(glob.glob('/content/processed/*0.png'))
# images = sorted(select_video, key=lambda x: int(x.split('attn')[0].split('/')[-1]))

In [33]:
# video_name = 'video.avi'

# frame = cv2.imread(images[0])
# height, width, layers = frame.shape

# video = cv2.VideoWriter(video_name, 0, 1, (width,height))

# for image in images:
#     video.write(cv2.imread(image))

# cv2.destroyAllWindows()
# video.release()

In [1]:
%%capture
!pip install moviepy

In [12]:
def images_to_video(fps, extension:str, video_name:str, output_format:str):

    import os
    import moviepy.video.io.ImageSequenceClip
    import glob

    select_video = sorted(glob.glob('/content/processed/processed/*0.png'))
    images = sorted(select_video, key=lambda x: int(x.split('attn')[0].split('/')[-1]))
    movie_clip = moviepy.video.io.ImageSequenceClip.ImageSequenceClip(images, fps)
    movie_clip.write_videofile(video_name+output_format)

In [13]:
images_to_video(54, '.jpeg', 'Spectra_video', '.mp4');

Moviepy - Building video Spectra_video.mp4.
Moviepy - Writing video Spectra_video.mp4



Moviepy - Done !
Moviepy - video ready Spectra_video.mp4
